In [4]:
import rasterio
import numpy as np
import subprocess

# Paths
ndvi_path = r"E:\DEMs\Headwaters_Boyer_River\8bit_NDVI.tif"
c_factor_tif = r"E:\DEMs\Headwaters_Boyer_River\C_Factor_float32.tif"
c_factor_rst = r"E:\DEMs\Headwaters_Boyer_River\C_Factor_float32.rst"

with rasterio.open(ndvi_path) as src:
    ndvi_int = src.read(1).astype(np.float32)
    ndvi_nodata = src.nodata  # Get original NoData value (e.g., 0 or 255)
    profile = src.profile.copy()
    profile.update(dtype=rasterio.float32, nodata=0)
    
    # Create mask for NoData cells
    if ndvi_nodata is not None:
        nodata_mask = (ndvi_int == ndvi_nodata)
    else:
        nodata_mask = np.zeros_like(ndvi_int, dtype=bool)
    
    ndvi = ndvi_int / 100.0
    alpha = 2.0
    c_factor = np.exp(-alpha * ndvi)
    c_factor = np.clip(c_factor, 0, 1)
    # Set nodata to 0 for WaTEM/SEDEM
    c_factor[nodata_mask] = 0

    with rasterio.open(c_factor_tif, 'w', **profile) as dst:
        dst.write(c_factor, 1)

# Convert GeoTIFF to RST format using gdal_translate
subprocess.run([
    'gdal_translate',
    '-of', 'RST',
    '-a_nodata', '0',
    c_factor_tif,
    c_factor_rst
])

print("C-factor raster saved at:", c_factor_rst)


C-factor raster saved at: E:\DEMs\Headwaters_Boyer_River\C_Factor_float32.rst
